In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

In [ ]:
#Create DataFrame where total_votes count is equal to or greater than 20
total_votes_20_df = df.filter(df.total_votes >= 20)
total_votes_20_df.show()

In [ ]:
#Create DataFrame where helpful_votes divided by total_votes is equal to or greater than 50%
helpful_votes_50_df = total_votes_20_df.filter((total_votes_20_df.helpful_votes / total_votes_20_df.total_votes) >= 0.5)
helpful_votes_50_df.show()

In [ ]:
#Create DataFrame where review was written as part of the Vine program (paid)
vine_yes_df = helpful_votes_50_df.filter(helpful_votes_50_df.vine == "Y")
vine_yes_df.show()

In [ ]:
#Create DtaFrame where the review was not part of the Vine program (unpaid)
vine_no_df = helpful_votes_50_df.filter(helpful_votes_50_df.vine == "N")
vine_no_df.show()

In [ ]:
#Total number of reviews
total_reviews = df.count()
total_reviews

In [ ]:
#Total number of 5-star reviews
total_5star_reviews = df.filter(df.star_rating == 5).count()
total_5star_reviews

In [ ]:
#Total of helpful reviews
total_helpful_reviews = helpful_votes_50_df.count()
total_helpful_reviews

In [ ]:
#Total number of helpful paid reviews
total_paid_reviews = vine_yes_df.count()
total_paid_reviews

In [ ]:
#Total number of helpful unpaid reviews
total_unpaid_reviews = vine_no_df.count()
total_unpaid_reviews

In [ ]:
#Total number of helpful 5-star reviews
total_helpful_5star_reviews = helpful_votes_50_df.filter(helpful_votes_50_df.star_rating == 5).count()
total_helpful_5star_reviews

In [ ]:
#Total number of helpful paid 5-star reviews
total_paid_5star_reviews = vine_yes_df.filter(vine_yes_df.star_rating == 5).count()
total_paid_5star_reviews

In [ ]:
#Total number of helpful unpaid 5-star reviews
total_unpaid_5star_reviews = vine_no_df.filter(vine_no_df.star_rating == 5).count()
total_unpaid_5star_reviews

In [ ]:
#Percentage of paid 5-star reviews
percent_paid_5star_reviews = (total_paid_5star_reviews / total_paid_reviews) * 100
percent_paid_5star_reviews

In [ ]:
#Percentage of unpaid 5-star reviews
percent_unpaid_5star_reviews = (total_unpaid_5star_reviews / total_unpaid_reviews) * 100
percent_unpaid_5star_reviews